In [1]:
import numpy as np
import pandas as pd
import pickle
import xgboost
import os
from scipy.stats import ttest_1samp, ttest_ind, ttest_rel
from config import path_prefix_results
import warnings

warnings.filterwarnings('ignore')


## Create latex template for skull, head results

In [2]:
#latex output setup
latex_template = r'''\documentclass[a4paper]{{article}}
\usepackage{{booktabs}}
\usepackage{{pdflscape}}
\begin{{document}}
\thispagestyle{{empty}}
\begin{{landscape}}
{}
\end{{landscape}}
\end{{document}}
'''

path_head_r2_latex= os.path.join(path_prefix_results, 'head_r2.tex')
path_head_rmse_latex= os.path.join(path_prefix_results, 'head_rmse.tex')
path_head_ttest_latex= os.path.join(path_prefix_results, 'head_ttest.tex')
path_head_fs_ttest_diff_latex= os.path.join(path_prefix_results, 'head_fs_ttest_diff.tex')
path_head_fs_ttest_diff_cross_latex= os.path.join(path_prefix_results, 'head_fs_ttest_diff_cross.tex')
path_skull_r2_latex= os.path.join(path_prefix_results, 'skull_r2.tex')
path_skull_rmse_latex= os.path.join(path_prefix_results, 'skull_rmse.tex')
path_skull_ttest_latex= os.path.join(path_prefix_results, 'skull_ttest.tex')
path_skull_fs_ttest_diff_latex= os.path.join(path_prefix_results, 'skull_fs_ttest_diff.tex')
path_skull_fs_ttest_diff_cross_latex= os.path.join(path_prefix_results, 'skull_fs_ttest_diff_cross.tex')

In [3]:
def test_diff_cross(df, row='multi-atlas'):
    tmp= pd.DataFrame(index=df.columns, columns=df.columns)
    for i in df.columns:
        for j in df.columns:
            tmp.loc[i,j]= ttest_rel(np.abs(df.loc[row, i]), np.abs(df.loc[row, j]))[1]
    return tmp

def test_diff(df, col='multi-atlas'):
    tmp= pd.DataFrame(index=df.index, columns=df.columns)
    for i in df.index:
        for j in df.columns:
            tmp.loc[i,j]= ttest_rel(np.abs(df.loc[i, j]), np.abs(df.loc[col, j]))[1]
    tmp= tmp.drop(col, axis='rows')
    return tmp

def test_compare(df0, df1):
    tmp= pd.DataFrame(index=df0.index, columns=df0.columns)
    for i in df0.index:
        for j in df0.columns:
            tmp.loc[i,j]= ttest_rel(np.abs(df0.loc[i, j]), np.abs(df1.loc[i, j]))[1]
    return tmp

## Load data 

In [4]:
data_fs= pickle.load(open(os.path.join(path_prefix_results,'results.pickle'), 'rb'))
data_no_fs= pickle.load(open(os.path.join(path_prefix_results,'results_no_fs.pickle'), 'rb'))

In [5]:
skull_fs= data_fs[data_fs['dataset'] == 'skull']
head_fs= data_fs[data_fs['dataset'] == 'head']

skull_no_fs= data_no_fs[data_no_fs['dataset'] == 'skull']
head_no_fs= data_no_fs[data_no_fs['dataset'] == 'head']

In [6]:
print(sorted(skull_fs['type'].unique()))

['127a', '130a', '139k', '152k', '166a', '234a', '237f', '240k', '246f', '261a', '274f', '275k', '284f', '297a', '298a', '305a', '310a', '316k', '319k', '333k', 'all', 'mean_mask']


In [7]:
rename_dict_regr= {'KNNR_Objective': 'kNN', 'LinearRegression_Objective': 'linear', 'PLSRegression_Objective': 'PLS', 'RidgeRegression_Objective': 'ridge', 'LassoRegression_Objective': 'lasso'}
#rename_dict_type= {'203a': 'atlas 1', '203k': 'atlas 2', '204f': 'atlas 3', '206k': 'atlas 4', '208f': 'atlas 5', 'all': 'multi-atlas', 'mean_mask': 'mean'}
#row_order= ['atlas 1', 'atlas 2', 'atlas 3', 'atlas 4', 'atlas 5', 'mean', 'multi-atlas']
rename_dict_type= sorted(skull_fs['type'].unique())
rename_dict_type= dict(map(lambda x, i: (x, f"atlas {i+1}"), rename_dict_type,  range(len(rename_dict_type))))
rename_dict_type['all']= "multi-atlas"
rename_dict_type['mean_mask'] = "mean"
print(rename_dict_type)
row_order= list(rename_dict_type.values())
col_order= ['linear', 'PLS', 'lasso', 'ridge', 'kNN']

{'127a': 'atlas 1', '130a': 'atlas 2', '139k': 'atlas 3', '152k': 'atlas 4', '166a': 'atlas 5', '234a': 'atlas 6', '237f': 'atlas 7', '240k': 'atlas 8', '246f': 'atlas 9', '261a': 'atlas 10', '274f': 'atlas 11', '275k': 'atlas 12', '284f': 'atlas 13', '297a': 'atlas 14', '298a': 'atlas 15', '305a': 'atlas 16', '310a': 'atlas 17', '316k': 'atlas 18', '319k': 'atlas 19', '333k': 'atlas 20', 'all': 'multi-atlas', 'mean_mask': 'mean'}


## skull $r^2$ results

In [8]:
skull_fs_results= skull_fs.pivot(index='type', columns='model', values='r2_1')
skull_fs_results= skull_fs_results.rename(rename_dict_regr, axis='columns')
skull_fs_results= skull_fs_results.rename(rename_dict_type)
skull_fs_results= skull_fs_results.apply(lambda x: np.round(x, 4))
skull_fs_results= skull_fs_results.loc[row_order, col_order]

skull_no_fs_results= skull_no_fs.pivot(index='type', columns='model', values='r2_1')
skull_no_fs_results= skull_no_fs_results.rename(rename_dict_regr, axis='columns')
skull_no_fs_results= skull_no_fs_results.rename(rename_dict_type)
skull_no_fs_results= skull_no_fs_results.apply(lambda x: np.round(x, 4))
skull_no_fs_results= skull_no_fs_results.loc[row_order, col_order]

skull_all= pd.concat([skull_fs_results, skull_no_fs_results], axis='columns')
columns= pd.MultiIndex.from_product([['feature selection', 'no feature selection'], col_order])
skull_all.columns= columns

In [9]:
skull_all_r2_latex= skull_all.to_latex()
print(skull_all_r2_latex)

#save results to tex file
with open(path_skull_r2_latex, 'w') as f:
    f.write(latex_template.format(skull_all_r2_latex))

\begin{tabular}{lrrrrrrrrrr}
\toprule
{} & \multicolumn{5}{l}{feature selection} & \multicolumn{5}{l}{no feature selection} \\
{} &            linear &     PLS &   lasso &   ridge &     kNN &               linear &     PLS &   lasso &   ridge &     kNN \\
type        &                   &         &         &         &         &                      &         &         &         &         \\
\midrule
atlas 1     &            0.9796 &  0.5478 &  0.5374 &  0.9819 &  0.7320 &              -0.1286 &  0.1560 &  0.1561 &  0.0903 &  0.3326 \\
atlas 2     &            0.4594 &  0.6299 &  0.5854 &  0.6434 &  0.5601 &              -0.1915 & -0.1125 &  0.1843 &  0.0043 &  0.3559 \\
atlas 3     &            0.7744 &  0.7102 &  0.7602 &  0.5821 &  0.6766 &              -0.3353 & -0.0799 &  0.2181 &  0.0403 &  0.1346 \\
atlas 4     &            0.7120 &  0.6920 &  0.7129 &  0.7014 &  0.7482 &              -0.2395 &  0.2179 &  0.3162 &  0.2526 &  0.2038 \\
atlas 5     &            0.3199 &  0.5970 &  

## skull RMSE results

In [10]:
skull_fs_results_rmse= skull_fs.pivot(index='type', columns='model', values='rmse_1')
skull_fs_results_rmse= skull_fs_results_rmse.rename(rename_dict_regr, axis='columns')
skull_fs_results_rmse= skull_fs_results_rmse.rename(rename_dict_type)
skull_fs_results_rmse= skull_fs_results_rmse.apply(lambda x: np.round(x, 4))
skull_fs_results_rmse= skull_fs_results_rmse.loc[row_order, col_order]

skull_no_fs_results_rmse= skull_no_fs.pivot(index='type', columns='model', values='rmse_1')
skull_no_fs_results_rmse= skull_no_fs_results_rmse.rename(rename_dict_regr, axis='columns')
skull_no_fs_results_rmse= skull_no_fs_results_rmse.rename(rename_dict_type)
skull_no_fs_results_rmse= skull_no_fs_results_rmse.apply(lambda x: np.round(x, 4))
skull_no_fs_results_rmse= skull_no_fs_results_rmse.loc[row_order, col_order]

skull_all_rmse= pd.concat([skull_fs_results_rmse, skull_no_fs_results_rmse], axis='columns')
columns= pd.MultiIndex.from_product([['feature selection', 'no feature selection'], col_order])
skull_all_rmse.columns= columns

In [11]:
skull_all_rmse_latex= skull_all_rmse.to_latex()
print(skull_all_rmse_latex)

#save results to tex file
with open(path_skull_rmse_latex, 'w') as f:
    f.write(latex_template.format(skull_all_rmse_latex))

\begin{tabular}{lrrrrrrrrrr}
\toprule
{} & \multicolumn{5}{l}{feature selection} & \multicolumn{5}{l}{no feature selection} \\
{} &            linear &     PLS &   lasso &   ridge &     kNN &               linear &      PLS &    lasso &    ridge &      kNN \\
type        &                   &         &         &         &         &                      &          &          &          &          \\
\midrule
atlas 1     &            1.4128 &  6.6497 &  6.7256 &  1.3301 &  5.1189 &              10.5053 &   9.0846 &   9.0841 &   9.4316 &   8.0786 \\
atlas 2     &            7.2707 &  6.0159 &  6.3672 &  5.9054 &  6.5589 &              10.7943 &  10.4304 &   8.9309 &   9.8674 &   7.9362 \\
atlas 3     &            4.6974 &  5.3232 &  4.8426 &  6.3925 &  5.6234 &              11.4270 &  10.2763 &   8.7441 &   9.6877 &   9.1991 \\
atlas 4     &            5.3067 &  5.4883 &  5.2990 &  5.4036 &  4.9625 &              11.0095 &   8.7450 &   8.1773 &   8.5488 &   8.8238 \\
atlas 5     &        

## t-test skull

In [12]:
skull_fs['diff']= (skull_fs['y_pred_1'] - skull_fs['y_test_1'])**2
skull_fs_test= skull_fs.pivot(index='type', columns='model', values='diff')
skull_fs_test= skull_fs_test.rename(rename_dict_regr, axis='columns')
skull_fs_test= skull_fs_test.rename(rename_dict_type)
skull_fs_test= skull_fs_test.loc[row_order, col_order]

skull_no_fs['diff']= (skull_no_fs['y_pred_1'] - skull_no_fs['y_test_1'])**2
skull_no_fs_test= skull_no_fs.pivot(index='type', columns='model', values='diff')
skull_no_fs_test= skull_no_fs_test.rename(rename_dict_regr, axis='columns')
skull_no_fs_test= skull_no_fs_test.rename(rename_dict_type)
skull_no_fs_test= skull_no_fs_test.loc[row_order, col_order]

In [13]:
skull_ttest_latex= test_compare(skull_fs_test, skull_no_fs_test).applymap(lambda x: '%.0e' % x).to_latex()
print(skull_ttest_latex)

#save results to tex file
with open(path_skull_ttest_latex, 'w') as f:
    f.write(latex_template.format(skull_ttest_latex))

\begin{tabular}{llllll}
\toprule
model & linear &    PLS &  lasso &  ridge &    kNN \\
type        &        &        &        &        &        \\
\midrule
atlas 1     &  7e-43 &  2e-40 &  8e-30 &  3e-62 &  1e-30 \\
atlas 2     &  3e-32 &  2e-30 &  1e-30 &  2e-33 &  3e-07 \\
atlas 3     &  2e-45 &  8e-61 &  8e-54 &  1e-34 &  2e-44 \\
atlas 4     &  1e-44 &  6e-38 &  1e-40 &  1e-41 &  6e-36 \\
atlas 5     &  5e-26 &  3e-32 &  4e-39 &  1e-43 &  6e-11 \\
atlas 6     &  1e-39 &  2e-47 &  1e-37 &  4e-40 &  3e-30 \\
atlas 7     &  4e-42 &  1e-21 &  5e-26 &  2e-34 &  3e-27 \\
atlas 8     &  2e-41 &  4e-35 &  8e-38 &  2e-37 &  9e-41 \\
atlas 9     &  7e-53 &  6e-38 &  1e-21 &  8e-28 &  9e-48 \\
atlas 10    &  2e-14 &  7e-29 &  2e-38 &  3e-23 &  1e-15 \\
atlas 11    &  5e-35 &  4e-25 &  2e-33 &  1e-18 &  2e-35 \\
atlas 12    &  1e-42 &  4e-38 &  6e-22 &  1e-49 &  4e-24 \\
atlas 13    &  7e-54 &  2e-42 &  8e-15 &  1e-32 &  1e-19 \\
atlas 14    &  6e-49 &  9e-25 &  4e-15 &  5e-39 &  6e-36 \\
atla

In [14]:
skull_fs_ttest_diff_latex= test_diff(skull_fs_test).applymap(lambda x: '%.0e' % x).to_latex()
print(skull_fs_ttest_diff_latex)

#save results to tex file
with open(path_skull_fs_ttest_diff_latex, 'w') as f:
    f.write(latex_template.format(skull_fs_ttest_diff_latex))

\begin{tabular}{llllll}
\toprule
model & linear &    PLS &  lasso &  ridge &    kNN \\
type     &        &        &        &        &        \\
\midrule
atlas 1  &  3e-31 &  2e-58 &  6e-65 &  9e-07 &  8e-01 \\
atlas 2  &  7e-27 &  3e-65 &  2e-58 &  3e-62 &  4e-10 \\
atlas 3  &  3e-01 &  1e-53 &  6e-66 &  5e-55 &  5e-04 \\
atlas 4  &  9e-06 &  8e-43 &  3e-34 &  3e-42 &  3e-01 \\
atlas 5  &  2e-17 &  1e-37 &  2e-42 &  1e-42 &  3e-39 \\
atlas 6  &  5e-51 &  2e-61 &  2e-36 &  1e-55 &  1e-04 \\
atlas 7  &  1e-42 &  2e-64 &  2e-72 &  5e-54 &  6e-08 \\
atlas 8  &  2e-19 &  4e-61 &  1e-53 &  5e-48 &  4e-01 \\
atlas 9  &  4e-29 &  6e-83 &  3e-64 &  1e-64 &  2e-04 \\
atlas 10 &  2e-15 &  2e-56 &  5e-43 &  5e-57 &  3e-15 \\
atlas 11 &  3e-56 &  7e-83 &  4e-72 &  7e-67 &  1e-05 \\
atlas 12 &  1e-30 &  2e-86 &  2e-74 &  6e-47 &  5e-25 \\
atlas 13 &  2e-34 &  3e-81 &  8e-70 &  1e-75 &  7e-47 \\
atlas 14 &  1e-22 &  3e-69 &  5e-65 &  1e-55 &  4e-37 \\
atlas 15 &  8e-10 &  1e-35 &  3e-36 &  1e-59 &  9

In [15]:
skull_fs_ttest_diff_cross_latex= test_diff_cross(skull_fs_test.loc[['multi-atlas']]).applymap(lambda x: '%.0e' % x).to_latex()
print(skull_fs_ttest_diff_cross_latex)

#save results to tex file
with open(path_skull_fs_ttest_diff_cross_latex, 'w') as f:
    f.write(latex_template.format(skull_fs_ttest_diff_cross_latex))

\begin{tabular}{llllll}
\toprule
model & linear &    PLS &  lasso &  ridge &    kNN \\
model  &        &        &        &        &        \\
\midrule
linear &    nan &  2e-34 &  6e-26 &  9e-30 &  1e-03 \\
PLS    &  2e-34 &    nan &  2e-26 &  4e-17 &  2e-53 \\
lasso  &  6e-26 &  2e-26 &    nan &  3e-02 &  1e-43 \\
ridge  &  9e-30 &  4e-17 &  3e-02 &    nan &  1e-45 \\
kNN    &  1e-03 &  2e-53 &  1e-43 &  1e-45 &    nan \\
\bottomrule
\end{tabular}



## head $r^2$ results

In [16]:
head_fs_results= head_fs.pivot(index='type', columns='model', values='r2_1')
head_fs_results= head_fs_results.rename(rename_dict_regr, axis='columns')
head_fs_results= head_fs_results.rename(rename_dict_type)
head_fs_results= head_fs_results.apply(lambda x: np.round(x, 4))
head_fs_results= head_fs_results.loc[row_order, col_order]

head_no_fs_results= head_no_fs.pivot(index='type', columns='model', values='r2_1')
head_no_fs_results= head_no_fs_results.rename(rename_dict_regr, axis='columns')
head_no_fs_results= head_no_fs_results.rename(rename_dict_type)
head_no_fs_results= head_no_fs_results.apply(lambda x: np.round(x, 4))
head_no_fs_results= head_no_fs_results.loc[row_order, col_order]

head_all= pd.concat([head_fs_results, head_no_fs_results], axis='columns')
columns= pd.MultiIndex.from_product([['feature selection', 'no feature selection'], col_order])
head_all.columns= columns

In [17]:
head_all_r2_latex= head_all.to_latex()
print(head_all_r2_latex)

#save results to tex file
with open(path_head_r2_latex, 'w') as f:
    f.write(latex_template.format(head_all_r2_latex))

\begin{tabular}{lrrrrrrrrrr}
\toprule
{} & \multicolumn{5}{l}{feature selection} & \multicolumn{5}{l}{no feature selection} \\
{} &            linear &     PLS &   lasso &   ridge &     kNN &               linear &     PLS &   lasso &   ridge &     kNN \\
type        &                   &         &         &         &         &                      &         &         &         &         \\
\midrule
atlas 1     &            0.7462 &  0.6899 &  0.6410 &  0.7488 &  0.6277 &              -0.3036 &  0.0656 &  0.1230 &  0.0266 &  0.2862 \\
atlas 2     &            0.8450 &  0.5938 &  0.8066 &  0.8354 &  0.7292 &              -0.1614 &  0.0807 &  0.0744 &  0.1389 &  0.3471 \\
atlas 3     &            0.6172 &  0.7147 &  0.4644 &  0.7208 &  0.6635 &              -0.7484 & -0.0095 &  0.1017 &  0.0276 &  0.3788 \\
atlas 4     &            0.6538 &  0.6850 &  0.4783 &  0.8239 &  0.7193 &              -0.6151 &  0.1400 &  0.2420 &  0.0597 &  0.3641 \\
atlas 5     &            0.6589 &  0.8176 &  

## head RMSE results

In [18]:
head_fs_results_rmse= head_fs.pivot(index='type', columns='model', values='rmse_1')
head_fs_results_rmse= head_fs_results_rmse.rename(rename_dict_regr, axis='columns')
head_fs_results_rmse= head_fs_results_rmse.rename(rename_dict_type)
head_fs_results_rmse= head_fs_results_rmse.apply(lambda x: np.round(x, 4))
head_fs_results_rmse= head_fs_results_rmse.loc[row_order, col_order]

head_no_fs_results_rmse= head_no_fs.pivot(index='type', columns='model', values='rmse_1')
head_no_fs_results_rmse= head_no_fs_results_rmse.rename(rename_dict_regr, axis='columns')
head_no_fs_results_rmse= head_no_fs_results_rmse.rename(rename_dict_type)
head_no_fs_results_rmse= head_no_fs_results_rmse.apply(lambda x: np.round(x, 4))
head_no_fs_results_rmse= head_no_fs_results_rmse.loc[row_order, col_order]

head_all_rmse= pd.concat([head_fs_results_rmse, head_no_fs_results_rmse], axis='columns')
columns= pd.MultiIndex.from_product([['feature selection', 'no feature selection'], col_order])
head_all_rmse.columns= columns

In [19]:
head_all_rmse_latex= head_all_rmse.to_latex()
print(head_all_rmse_latex)

#save results to tex file
with open(path_head_rmse_latex, 'w') as f:
    f.write(latex_template.format(head_all_rmse_latex))

\begin{tabular}{lrrrrrrrrrr}
\toprule
{} & \multicolumn{5}{l}{feature selection} & \multicolumn{5}{l}{no feature selection} \\
{} &            linear &     PLS &   lasso &   ridge &     kNN &               linear &      PLS &   lasso &    ridge &     kNN \\
type        &                   &         &         &         &         &                      &          &         &          &         \\
\midrule
atlas 1     &            4.9814 &  5.5070 &  5.9248 &  4.9567 &  6.0340 &              11.2907 &   9.5588 &  9.2606 &   9.7563 &  8.3549 \\
atlas 2     &            3.8930 &  6.3029 &  4.3491 &  4.0125 &  5.1463 &              10.6571 &   9.4813 &  9.5139 &   9.1762 &  7.9903 \\
atlas 3     &            6.1183 &  5.2823 &  7.2373 &  5.2252 &  5.7364 &              13.0758 &   9.9359 &  9.3726 &   9.7512 &  7.7938 \\
atlas 4     &            5.8185 &  5.5500 &  7.1424 &  4.1496 &  5.2390 &              12.5672 &   9.1704 &  8.6097 &   9.5888 &  7.8855 \\
atlas 5     &            5.7755 &

## t-test head

In [20]:
head_fs['diff']= (head_fs['y_pred_1'] - head_fs['y_test_1'])**2
head_fs_test= head_fs.pivot(index='type', columns='model', values='diff')
head_fs_test= head_fs_test.rename(rename_dict_regr, axis='columns')
head_fs_test= head_fs_test.rename(rename_dict_type)
head_fs_test= head_fs_test.loc[row_order, col_order]

head_no_fs['diff']= (head_no_fs['y_pred_1'] - head_no_fs['y_test_1'])**2
head_no_fs_test= head_no_fs.pivot(index='type', columns='model', values='diff')
head_no_fs_test= head_no_fs_test.rename(rename_dict_regr, axis='columns')
head_no_fs_test= head_no_fs_test.rename(rename_dict_type)
head_no_fs_test= head_no_fs_test.loc[row_order, col_order]

In [21]:
head_ttest_latex= test_compare(head_fs_test, head_no_fs_test).applymap(lambda x: '%.0e' % x).to_latex()
print(head_ttest_latex)

#save results to tex file
with open(path_head_ttest_latex, 'w') as f:
    f.write(latex_template.format(head_ttest_latex))

\begin{tabular}{llllll}
\toprule
model & linear &    PLS &  lasso &  ridge &    kNN \\
type        &        &        &        &        &        \\
\midrule
atlas 1     &  6e-41 &  5e-44 &  8e-51 &  8e-63 &  9e-16 \\
atlas 2     &  4e-41 &  3e-48 &  6e-65 &  1e-50 &  1e-23 \\
atlas 3     &  5e-49 &  5e-47 &  4e-30 &  5e-48 &  3e-18 \\
atlas 4     &  1e-55 &  5e-37 &  3e-18 &  6e-59 &  3e-23 \\
atlas 5     &  1e-38 &  5e-39 &  7e-81 &  3e-41 &  9e-28 \\
atlas 6     &  1e-38 &  1e-52 &  1e-30 &  2e-47 &  3e-45 \\
atlas 7     &  7e-44 &  6e-44 &  2e-39 &  5e-49 &  5e-48 \\
atlas 8     &  1e-61 &  1e-57 &  8e-17 &  2e-54 &  6e-27 \\
atlas 9     &  2e-18 &  2e-35 &  2e-34 &  7e-33 &  2e-41 \\
atlas 10    &  1e-33 &  7e-61 &  8e-44 &  7e-69 &  5e-46 \\
atlas 11    &  3e-40 &  1e-51 &  3e-40 &  6e-54 &  2e-37 \\
atlas 12    &  6e-32 &  2e-37 &  1e-38 &  4e-39 &  2e-31 \\
atlas 13    &  4e-23 &  1e-55 &  6e-15 &  2e-48 &  2e-37 \\
atlas 14    &  5e-43 &  1e-32 &  9e-24 &  2e-37 &  1e-33 \\
atla

In [22]:
head_fs_ttest_diff_latex= test_diff(head_fs_test).applymap(lambda x: '%.0e' % x).to_latex()
print(head_fs_ttest_diff_latex)

#save results to tex file
with open(path_head_fs_ttest_diff_latex, 'w') as f:
    f.write(latex_template.format(head_fs_ttest_diff_latex))

\begin{tabular}{llllll}
\toprule
model &  linear &     PLS &  lasso &  ridge &    kNN \\
type     &         &         &        &        &        \\
\midrule
atlas 1  &   4e-60 &   3e-61 &  7e-37 &  1e-67 &  2e-05 \\
atlas 2  &   1e-60 &   2e-78 &  6e-36 &  5e-80 &  6e-01 \\
atlas 3  &   6e-53 &   2e-93 &  2e-76 &  3e-97 &  1e-02 \\
atlas 4  &   5e-46 &   3e-46 &  3e-72 &  2e-36 &  9e-01 \\
atlas 5  &   4e-81 &   3e-41 &  4e-33 &  2e-61 &  2e-02 \\
atlas 6  &   4e-81 &   4e-69 &  5e-61 &  3e-77 &  7e-06 \\
atlas 7  &  5e-109 &   6e-95 &  6e-39 &  2e-94 &  2e-08 \\
atlas 8  &   1e-59 &   4e-84 &  1e-75 &  2e-62 &  2e-07 \\
atlas 9  &   1e-71 &   8e-73 &  2e-68 &  1e-52 &  2e-02 \\
atlas 10 &   3e-46 &   3e-82 &  2e-47 &  5e-47 &  4e-11 \\
atlas 11 &   2e-71 &  3e-107 &  6e-53 &  2e-70 &  4e-14 \\
atlas 12 &   1e-57 &   4e-81 &  6e-51 &  2e-49 &  9e-01 \\
atlas 13 &   3e-84 &   1e-84 &  4e-67 &  6e-85 &  5e-18 \\
atlas 14 &   9e-54 &   3e-77 &  9e-68 &  1e-67 &  8e-30 \\
atlas 15 &   3e-8

In [23]:
head_fs_ttest_diff_cross_latex= test_diff_cross(head_fs_test.loc[['multi-atlas']]).applymap(lambda x: '%.0e' % x).to_latex()
print(head_fs_ttest_diff_cross_latex)

#save results to tex file
with open(path_head_fs_ttest_diff_cross_latex, 'w') as f:
    f.write(latex_template.format(head_fs_ttest_diff_cross_latex))

\begin{tabular}{llllll}
\toprule
model & linear &    PLS &  lasso &  ridge &    kNN \\
model  &        &        &        &        &        \\
\midrule
linear &    nan &  3e-03 &  2e-19 &  2e-32 &  9e-66 \\
PLS    &  3e-03 &    nan &  9e-28 &  2e-27 &  2e-66 \\
lasso  &  2e-19 &  9e-28 &    nan &  1e-44 &  4e-57 \\
ridge  &  2e-32 &  2e-27 &  1e-44 &    nan &  2e-68 \\
kNN    &  9e-66 &  2e-66 &  4e-57 &  2e-68 &    nan \\
\bottomrule
\end{tabular}



### Convert tex files to pdfs

In [24]:
import shutil
import glob

pdflatex_path= shutil.which("pdflatex")
if path_prefix_results != os.path.basename(os.getcwd()):
    os.chdir(path_prefix_results)
texs= glob.glob("*tex")
temp = "tmp"
os.makedirs(temp, exist_ok=True)

In [25]:
for tex in texs:
    print(f"Converting {tex}...")
    cmd= f"{pdflatex_path} {tex} >> /dev/null"
    os.system(cmd)
    for i in glob.glob("*.aux") + glob.glob("*.log"):
        shutil.move(os.path.abspath(i), os.path.join(os.path.abspath(temp),i))

Converting head_fs_ttest_diff.tex...
Converting head_rmse.tex...
Converting head_r2.tex...
Converting skull_ttest.tex...
Converting skull_fs_ttest_diff_cross.tex...
Converting skull_r2.tex...
Converting skull_rmse.tex...
Converting head_fs_ttest_diff_cross.tex...
Converting skull_fs_ttest_diff.tex...
Converting head_ttest.tex...
